In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
tf.reset_default_graph() #그래프 리셋

In [6]:
np.random.seed(20170821) #가중치를 위한 랜덤시드 생성

In [7]:
tf.set_random_seed(20170821)

In [8]:
mnist = input_data.read_data_sets("C:\\Users\\min\\Downloads\\cifar-10-python.tar.gz", one_hot=True) #데이터 셋 로드

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting C:\Users\min\Downloads\cifar-10-python.tar.gz\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting C:\Users\min\Downloads\cifar-10-python.tar.gz\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting C:\Users\min\Downloads\cifar-10-python.tar.gz\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting C:\Users\min\Downloads\cifar-10-python.tar.gz\t10k-labels-idx1-ubyte.gz


In [9]:
#1층 레이어 설계

In [10]:
num_filters1 = 32   #1차 필터 갯수 정의

In [11]:
x = tf.placeholder(tf.float32,[None, 784]) 

In [27]:
x_image = tf.reshape(x,[-1,28,28,1]) #이미지 데이터 x -> 28x28x1의 무한개 행렬 변환
print(x_image)

Tensor("Reshape_2:0", shape=(?, 28, 28, 1), dtype=float32)


In [28]:
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],stddev=0.1)) # truncated_normal : 임의의 수 지정
print(W_conv1)

<tf.Variable 'Variable_4:0' shape=(5, 5, 1, 32) dtype=float32_ref>


In [29]:
h_conv1 = tf.nn.conv2d(x_image,W_conv1,strides=[1,1,1,1],padding='SAME') #conv2d 필터 적용 / stride 방식 / 자동 패딩 삽입
print(h_conv1)

Tensor("Conv2D_2:0", shape=(?, 28, 28, 32), dtype=float32)


In [30]:
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1])) # 
print(b_conv1)

<tf.Variable 'Variable_5:0' shape=(32,) dtype=float32_ref>


In [31]:
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)
print(h_conv1_cutoff)

Tensor("Relu_2:0", shape=(?, 28, 28, 32), dtype=float32)


In [33]:
h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')
print(h_pool1)

Tensor("MaxPool_3:0", shape=(?, 14, 14, 32), dtype=float32)


In [18]:
#2층 레이어 설계

In [19]:
num_filters2 = 64

In [34]:
W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
print(W_conv2)

<tf.Variable 'Variable_6:0' shape=(5, 5, 32, 64) dtype=float32_ref>


In [35]:
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')
print(h_conv2)

Tensor("Conv2D_3:0", shape=(?, 14, 14, 64), dtype=float32)


In [36]:
b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
print(b_conv2)

<tf.Variable 'Variable_7:0' shape=(64,) dtype=float32_ref>


In [37]:
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)
print(h_conv2_cutoff)

Tensor("Relu_3:0", shape=(?, 14, 14, 64), dtype=float32)


In [24]:
h_pool2 = tf.nn.max_pool(h_conv2_cutoff,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [25]:
print(h_pool2,num_filters2)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32) 64


In [26]:
h_pool2_float = tf.reshape(h_pool2,[-1,7*7*num_filters2])
print(h_pool2_float)

Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)


In [26]:
num_units1 = 7*7*num_filters2
print(num_units1)

In [27]:
num_units2 = 1024

In [28]:
w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))

In [29]:
b2 = tf.Variable(tf.constant(0.1,shape=[num_units2]))

In [30]:
hidden2 = tf.nn.relu(tf.matmul(h_pool2_float,w2)+b2)

In [31]:
keep_prob = tf.placeholder(tf.float32)

In [32]:
hidden2_drop = tf.nn.dropout(hidden2,keep_prob)

In [33]:
w0 = tf.Variable(tf.zeros([num_units2,10]))

In [34]:
b0 = tf.Variable(tf.zeros([10]))

In [35]:
k = tf.matmul(hidden2_drop,w0)+b0

In [36]:
p = tf.nn.softmax(k)

In [37]:
t = tf.placeholder(tf.float32,[None, 10])

In [38]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = k,labels=t)) 

In [39]:
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [40]:
correct_prediction=tf.equal(tf.argmax(p,1),tf.argmax(t,1))

In [41]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [42]:
sess = tf.InteractiveSession()

In [43]:
sess.run(tf.global_variables_initializer())

In [44]:
saver = tf.train.Saver()

In [45]:
i = 0
for _ in range(10000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
#     if i % 500 == 0:
#         loss_vals, acc_vals = [],[]
#         for c in range(4):
#             start = len(mnist.test.labels) / 4 * c
#             end = len(mnist.test.labels) / 4 * (c+1)
#             loss_val, acc_val = sess.run([loss, accuracy],
#                 feed_dict={x:int(mnist.test.images[start:end]),
#                            t:int(mnist.test.labels[start:end]),
#                            keep_prob:1.0})
#             loss_vals.append(loss_val)
#             acc_vals.append(acc_val)
#         loss_val = np.sum(loss_vals)
#         acc_val = np.mean(acc_vals)
#         print ('Step: %d, Loss: %f, Accuracy: %f'
#                % (i, loss_val, acc_val))

In [46]:
saver.save(sess, './test_session/cnn_session')

'./test_session/cnn_session'

In [47]:
sess.close()